# DEA Coastlines vector statistics <img align="right" src="https://github.com/GeoscienceAustralia/dea-notebooks/raw/develop/Supplementary_data/dea_logo.jpg">

This code conducts vector subpixel shoreline extraction for DEA Coastlines:

* Apply morphological extraction algorithms to mask annual median composite rasters to a valid coastal region
* Extract waterline vectors using subpixel waterline extraction ([Bishop-Taylor et al. 2019b](https://doi.org/10.3390/rs11242984))
* Compute rates of coastal change at every 30 m using linear regression

This is an interactive version of the code intended for prototyping; to run this analysis at scale, use the [command line tools](DEACoastlines_generation_CLI.ipynb).


---

## Getting started
Set working directory to top level of repo to ensure links work correctly:

In [ ]:
cd ..

### Load packages

First we import the required Python packages, then we connect to the database, and load the catalog of virtual products.

In [ ]:
# pip install -r requirements.in --quiet

In [ ]:
%matplotlib inline
# %load_ext line_profiler
%load_ext autoreload
%autoreload 2

import os

# Disable PYGEOS
os.environ['USE_PYGEOS'] = '0'

import odc.geo.xr  # noqa 401
import numpy as np
import pandas as pd
import geohash as gh
import geopandas as gpd
from dea_tools.spatial import subpixel_contours

# Load DEA Coastlines code
import coastlines.raster
import coastlines.vector

# Hide Pandas warnings
pd.options.mode.chained_assignment = None


## Load in data

In [ ]:
config_path = 'configs/vietnam_coastlines_config_development.yaml'

# study_area = "13,45"
study_area = "9,19"

raster_version = 'testing'
vector_version = raster_version
start_year = 2020
end_year = 2022
baseline_year = 2021

water_index = "mndwi"
index_threshold = 0.25

# Load analysis params from config file
config = coastlines.raster.load_config(
    config_path=config_path
)

## Load DEA Coastlines rasters

In [ ]:
yearly_ds, gapfill_ds = coastlines.vector.load_rasters(
    path="data/interim/raster",
    raster_version=raster_version,
    study_area=study_area,
    water_index=water_index,
    start_year=start_year,
    end_year=end_year,
)

# Create output vector folder
output_dir = f"data/interim/vector/{vector_version}/{study_area}_{vector_version}"
os.makedirs(output_dir, exist_ok=True)

# Plot an image
yearly_ds.isel(year=0).mndwi.plot(cmap="RdBu")

## Load vector data

In [ ]:
# Get bounding box to load data for
bbox = gpd.GeoSeries(yearly_ds.odc.geobox.extent.geom, crs=yearly_ds.odc.crs)

# Study area polygon
gridcell_gdf = (
    gpd.read_file(config["Input files"]["grid_path"], bbox=bbox)
    .set_index("id")
    .to_crs(str(yearly_ds.odc.crs))
)
gridcell_gdf.index = gridcell_gdf.index.astype(str)
gridcell_gdf = gridcell_gdf.loc[[str(study_area)]]

# Coastal mask modifications
modifications_gdf = gpd.read_file(
    config["Input files"]["modifications_path"], bbox=bbox
).to_crs(str(yearly_ds.odc.crs))

# # Geomorphology dataset
# geomorphology_gdf = gpd.read_file(
#     config["Input files"]["geomorphology_path"], bbox=bbox
# ).to_crs(str(yearly_ds.odc.crs))

# # Region attribute dataset
# region_gdf = gpd.read_file(
#     config["Input files"]["region_attributes_path"], bbox=bbox
# ).to_crs(str(yearly_ds.odc.crs))

In [ ]:
# Mask dataset to focus on coastal zone only
masked_ds, certainty_masks = coastlines.vector.contours_preprocess(
    yearly_ds,
    gapfill_ds,
    water_index,
    index_threshold,
    mask_with_esa_wc=True,
    buffer_pixels=33,
    mask_modifications=modifications_gdf,
)

# Plot timestep
masked_ds.isel(year=0).plot()

In [ ]:
# Extract shorelines
contours_gdf = subpixel_contours(
    da=masked_ds,
    z_values=index_threshold,
    min_vertices=10,
    dim="year",
#     output_path=f"temp.geojson",
).set_index("year")

# Plot shorelines
contours_gdf.plot()

In [ ]:
import folium

# Stamen Terrain
bb = masked_ds.odc.geobox.boundingbox.to_crs(4326)
d_x = bb.right - bb.left
d_y = bb.top - bb.bottom

location = (bb.bottom + d_y / 2, bb.left + d_x / 2) 

map = folium.Map(
  location=location,
  tiles='https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
  attr='Esri',
  name='Esri Satellite',
  zoom_start=10
)

for _, y in contours_gdf.to_crs("epsg:4326").iterrows():
    for r in y:
      sim_geo = gpd.GeoSeries(r.geoms)
      geo_j = sim_geo.to_json()
      geo_j = folium.GeoJson(data=geo_j, style_function=lambda x: {"stroke": "red"})
      geo_j.add_to(map)
map

In [ ]:
# # DEBUG
# (
#     masked_ds,
#     certainty_masks,
#     all_time_15,
#     all_time_50,
#     river_mask,
#     ocean_da,
#     thresholded_ds,
#     temporal_mask,
#     annual_mask,
#     coastal_mask,
# ) = coastlines.vector.contours_preprocess(
#     yearly_ds,
#     gapfill_ds,
#     water_index,
#     index_threshold,
#     buffer_pixels=33,
#     mask_modifications=modifications_gdf,
#     debug=True,
# )

## Compute statistics
### Create stats points on baseline shoreline

In [ ]:
# Extract statistics modelling points along baseline shoreline
try:
    points_gdf = coastlines.vector.points_on_line(contours_gdf, baseline_year, distance=30)
except KeyError:
    print("Failed to find the baseline year")
    points_gdf = None

### Measure annual coastline movements

In [ ]:
if points_gdf is not None and len(points_gdf) > 0:
    
    # Calculate annual movements for every shoreline
    # compared to the baseline year
    points_gdf = coastlines.vector.annual_movements(
        points_gdf,
        contours_gdf,
        yearly_ds,
        str(baseline_year),
        water_index,
        max_valid_dist=1200,
    )
    
    # Reindex to add any missing annual columns to the dataset
    points_gdf = points_gdf.reindex(
        columns=[
            "geometry",
            *[f"dist_{i}" for i in range(start_year, end_year + 1)],
            "angle_mean",
            "angle_std",
        ]
    )

### Calculate regressions

In [ ]:
if points_gdf is not None and len(points_gdf) > 0:

    # Apply regression function to each row in dataset
    points_gdf = coastlines.vector.calculate_regressions(points_gdf)

    # Add count and span of valid obs, Shoreline Change Envelope (SCE),
    # Net Shoreline Movement (NSM) and Max/Min years
    stats_list = ["valid_obs", "valid_span", "sce", "nsm", "max_year", "min_year"]
    points_gdf[stats_list] = points_gdf.apply(
        lambda x: coastlines.vector.all_time_stats(x, initial_year=start_year), axis=1
    )

### Add certainty flags
Add certainty column to flag points with:
- Baseline outlier: The baseline shoreline is itself flagged as an outlier, potentially resulting in inaccurate rates of change.
- Likely rocky shorelines: Rates of change can be unreliable in areas with steep rocky/bedrock shorelines due to terrain shadow.
- Extreme rate of change value (> 50 m per year change) that is more likely to reflect modelling issues than real-world coastal change
- High angular variability: the nearest shorelines for each year do not fall on an approximate line, making rates of change invalid
- Insufficient observations: less than 75% valid annual shorelines, which make the resulting rates of change more likely to be inaccurate

In [ ]:
if points_gdf is not None and len(points_gdf) > 0:
    
    rocky = [
        "Bedrock breakdown debris (cobbles/boulders)",
        "Boulder (rock) beach",
        "Cliff (>5m) (undiff)",
        "Colluvium (talus) undiff",
        "Flat boulder deposit (rock) undiff",
        "Hard bedrock shore",
        "Hard bedrock shore inferred",
        "Hard rock cliff (>5m)",
        "Hard rocky shore platform",
        "Rocky shore (undiff)",
        "Rocky shore platform (undiff)",
        "Sloping hard rock shore",
        "Sloping rocky shore (undiff)",
        "Soft `bedrock¿ cliff (>5m)",
        "Steep boulder talus",
        "Hard rocky shore platform",
    ]

    # Initialise certainty column with good values
    points_gdf["certainty"] = "good"

    # Flag points where the baseline shoreline is itself an outlier
    points_gdf.loc[
        points_gdf.outl_time.str.contains(str(baseline_year)), "certainty"
    ] = "baseline outlier"

    # # Flag rocky shorelines
    # points_gdf.loc[
    #     coastlines.vector.rocky_shoreline_flag(
    #         points_gdf,
    #         geomorphology_gdf,
    #         rocky_query=f"(INTERTD1_V in {rocky}) & (INTERTD2_V in {rocky + ['Unclassified']})",
    #     ),
    #     "certainty",
    # ] = "likely rocky coastline"

    # Flag extreme rates of change
    points_gdf.loc[points_gdf.rate_time.abs() > 50, "certainty"] = "extreme value (> 50 m)"

    # Flag points where change does not fall on a line
    points_gdf.loc[points_gdf.angle_std > 30, "certainty"] = "high angular variability"

    # Flag points with less than X valid shorelines
    valid_obs_thresh = int(points_gdf.columns.str.contains("dist_").sum() * 0.75)
    points_gdf.loc[
        points_gdf.valid_obs < valid_obs_thresh, "certainty"
    ] = "insufficient observations"

### Add region attributes

In [ ]:
if points_gdf is not None and len(points_gdf) > 0:

    points_gdf = coastlines.vector.region_attributes(
        points_gdf, region_gdf, attribute_col="ID_Primary", rename_col="id_primary"
    )

### Add geohash unique IDs

In [ ]:
if points_gdf is not None and len(points_gdf) > 0:
    
    # Generate a geohash UID for each point and set as index
    uids = points_gdf.geometry.to_crs("EPSG:4326").apply(
        lambda x: gh.encode(x.y, x.x, precision=10)
    ).rename('uid')
    points_gdf = points_gdf.set_index(uids)

## Export files

### Export stats files

In [ ]:
if points_gdf is not None and len(points_gdf) > 0:

    # Clip stats to study area extent
    points_gdf_clipped = points_gdf.clip(gridcell_gdf)

    # Set output path
    stats_path = (
        f"{output_dir}/ratesofchange_{study_area}_"
        f"{vector_version}_{water_index}_{index_threshold:.2f}"
    )
    
    # Export to GeoJSON
    points_gdf_clipped.to_crs("EPSG:4326").to_file(
        f"{stats_path}.geojson",
        driver="GeoJSON",
    )

    # Export as ESRI shapefiles
    points_gdf_clipped.to_file(
        f"{stats_path}.shp",
        schema={
            "properties": coastlines.vector.vector_schema(points_gdf_clipped),
            "geometry": "Point",
        },
    )
    

### Export annual shorelines

In [ ]:
if len(contours_gdf.index) > 0:

    # # Assign certainty to contours based on underlying masks
    # contours_gdf = coastlines.vector.contour_certainty(contours_gdf, certainty_masks)

    # Add tide datum details (this supports future addition of extra tide datums)
    contours_gdf["tide_datum"] = "0.0 m AMSL"

    # # Add region attributes
    # contours_gdf = coastlines.vector.region_atttributes(
    #     contours_gdf, region_gdf, attribute_col="ID_Primary", rename_col="id_primary"
    # )

    # Set output path
    contour_path = (
        f"{output_dir}/annualshorelines_{study_area}_{vector_version}_"
        f"{water_index}_{index_threshold:.2f}"
    )

    # Clip annual shoreline contours to study area extent
    contours_gdf_clipped = contours_gdf.clip(gridcell_gdf)

    # Export to GeoJSON
    contours_gdf_clipped.to_crs("EPSG:4326").to_file(
        f"{contour_path}.geojson", driver="GeoJSON"
    )

    # Export stats and contours as ESRI shapefiles
    contours_gdf_clipped.to_file(
        f"{contour_path}.shp",
        schema={
            "properties": coastlines.vector.vector_schema(contours_gdf_clipped),
            "geometry": ["MultiLineString", "LineString"],
        },
    )

***

## Additional information

**License:** The code in this notebook is licensed under the [Apache License, Version 2.0](https://www.apache.org/licenses/LICENSE-2.0). 
Digital Earth Australia data is licensed under the [Creative Commons by Attribution 4.0](https://creativecommons.org/licenses/by/4.0/) license.

**Contact:** For assistance with any of the Python code or Jupyter Notebooks in this repository, please post a [Github issue](https://github.com/GeoscienceAustralia/dea-coastlines/issues/new).

**Last modified:** November 2022

**To cite:**

> Bishop-Taylor, R., Nanson, R., Sagar, S., Lymburner, L. (2021). Mapping Australia's dynamic coastline at mean sea level using three decades of Landsat imagery. Remote Sensing of Environment, 267, 112734. Available: https://doi.org/10.1016/j.rse.2021.112734
>
> Nanson, R., Bishop-Taylor, R., Sagar, S., Lymburner, L., (2022). Geomorphic insights into Australia's coastal change using a national dataset derived from the multi-decadal Landsat archive. Estuarine, Coastal and Shelf Science, 265, p.107712. Available: https://doi.org/10.1016/j.ecss.2021.107712
>
> Bishop-Taylor, R., Sagar, S., Lymburner, L., Alam, I., Sixsmith, J. (2019). Sub-pixel waterline extraction: characterising accuracy and sensitivity to indices and spectra. Remote Sensing, 11 (24):2984. Available: https://doi.org/10.3390/rs11242984